In [2]:
import spotipy
import spotipy.util as util
import numpy as np
import pandas as pd

In [3]:
absolute_path = "f:\\AI Projects\\Music Classification"
relative_path = "Data\\Spotify"
full_path = os.path.join(absolute_path, relative_path)

In [4]:
CLIENT_ID = "372d9bfe8dce48a1aac1806d4329629d"
CLIENT_SECRET = "1a6e5a340428469dba631f9ffbd4528f"

In [5]:
auth_manager = spotipy.oauth2.SpotifyClientCredentials(client_id=CLIENT_ID,client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [6]:
def get_playlist_tracks(playlist_id):
    playlist_data = sp.playlist_tracks(playlist_id)
    tracks = playlist_data['items']
    while playlist_data['next']:
        playlist_data = sp.next(playlist_data)
        tracks.extend(playlist_data['items'])
        
    return tracks

Get tracks from playlists

In [9]:
playlists = ["2MepDCzimtCz2ytAlLUdUi","1h0CEZCm6IbFTbxThn6Xcs","37i9dQZF1DX8qqIDAkKiQg","7pWNG89bI9uwHQaUI3pINb","37i9dQZF1DX12YIlzNkccL","2hRc6PoqD046cyS8QTFKkQ"] #["4rnleEAOdmFAbRcNCgZMpY","37i9dQZF1DWWEJlAGA9gs0","37i9dQZF1DXbITWG1ZJKYt","37i9dQZF1DX1spT6G94GFC","5oM9tyhspzK18TrPrlIZro","37i9dQZF1DX55yuR78Invt"]

In [11]:
import warnings
warnings.filterwarnings('ignore')

columns = ["artist","album","track_name","track_id","released_date","added_date","genres","danceability","energy","key","loudness","mode","speechiness","acousticness","instrumentalness","liveness","valence","tempo","duration_ms","time_signature"]

metadata_playlists = pd.DataFrame(columns=columns)

for playlist in playlists:
    playlist_data = get_playlist_tracks(playlist)
    for track in playlist_data:
        artist = track["track"]["album"]["artists"][0]["name"]
        artist_uri = track["track"]["album"]["artists"][0]["uri"]
        album = track["track"]["album"]["name"]
        track_name = track["track"]["name"]
        if not (
            metadata_playlists['artist'].str.contains(artist, regex=False).any() & 
            metadata_playlists['album'].str.contains(album, regex=False).any() & 
            metadata_playlists['track_name'].str.contains(track_name, regex=False).any()):
            track_id = track["track"]["id"]
            released_date = track["track"]["album"]["release_date"]
            added_date = track["added_at"]
            genres = sp.artist(artist_uri)["genres"]
            new_row = {"artist" : artist, "album" : album, "track_name" : track_name, "track_id" : track_id,"released_date" : released_date, "added_date" : added_date,"genres" : genres}
            audio_features = sp.audio_features(track_id)[0]
            for feature in columns[7:]:
                new_row[feature] = audio_features[feature]
            metadata_playlists = metadata_playlists.append(new_row, ignore_index=True)
        else:
            print(f"Skipping: {track_name} due to a duplicate")


Every Breath You Take
Sweet Child O' Mine
Too Many Pieces
Your Love
Africa
Gucci
Livin' On A Prayer
The Final Countdown
Eye of the Tiger
Nothing's Gonna Stop Us Now
Back In Black
I Want to Know What Love Is - 1999 Remaster
Don't Stop Believin'
Patience
Another Day in Paradise - 2016 Remaster
Paradise City
Heaven
She's Like the Wind (feat. Wendy Fraser)
Welcome To The Jungle
Walk Of Life
I'm Still Standing
Don't You (Forget About Me) - Remastered 2001
I Still Haven't Found What I'm Looking For
Alone
Listen To Your Heart
You Shook Me All Night Long
Maneater
Burning Heart - From "Rocky IV" Soundtrack
Don't Dream It's Over
Never Tear Us Apart
Jump - 2015 Remaster
You Give Love A Bad Name
Carry on Wayward Son
Everytime You Go Away
Bette Davis Eyes
I Love Rock 'n Roll
Faithfully
Purple Rain
With Or Without You - Remastered
Shout
Tougher Than the Rest
Beds Are Burning - Remastered
Poison
How Am I Supposed to Live Without You
I Hate Myself for Loving You
Who Made Who
Can't Fight This Feeling
I

In [ ]:
metadata_playlists.head()

In [ ]:
metadata_playlists.to_csv(os.path.join(full_path,"metadata_playlists_1.csv"))

Define all artists

In [7]:
artists = [
"spotify:artist:36QJpDe2go2KgaRleHCDTp",
"spotify:artist:6mdiAmATAx73kdxrNrnlao",
"spotify:artist:2ye2Wgw4gimLv2eAKyk1NB",
"spotify:artist:58lV9VcRSjABbAbfWS6skp",
"spotify:artist:27T030eWyCQRmDyuvr1kxY",
"spotify:artist:26bcq2nyj5GB7uRr558iQg",
"spotify:artist:6olE6TJLqED3rqDCT0FyPh",
"spotify:artist:0rvjqX7ttXeg3mTy8Xscbt",
"spotify:artist:711MCceyCBcFnzjGY4Q7Un",
"spotify:artist:3qm84nBOXUEQ2vnTfUTTFC",
"spotify:artist:7Js6Lde8thlIHXggv2SCBz"
]

Get all tracks from all albums from artists

In [14]:
import warnings
warnings.filterwarnings('ignore')

columns = ["artist","album_name","track_name","track_id","released_date","explict","genres","danceability","energy","key","loudness","mode","speechiness","acousticness","instrumentalness","liveness","valence","tempo","duration_ms","time_signature"]

metadata_albums = pd.DataFrame(columns=columns)

for artist_uri in artists:
    artist = sp.artist(artist_uri)["name"]
    results = sp.artist_albums(artist_uri, album_type='album')
    albums = results['items']
    while results['next']:
        results = sp.next(results)
        albums.extend(results['items'])
    for album in albums:
        album_name = album["name"]
        released_date = album["release_date"]
        tracks = sp.album_tracks(album["id"])      
        for track in tracks["items"]:
            track_name = track["name"]
            if not (
                metadata_albums['artist'].str.contains(artist, regex=False).any() & 
                metadata_albums['album_name'].str.contains(album_name, regex=False).any() & 
                metadata_albums['track_name'].str.contains(track_name, regex=False).any()):
                
                track_id = track["id"]
                explict = track["explicit"]
                genres = sp.artist(artist_uri)["genres"]
                new_row = {"artist" : artist, "album_name" : album_name, "track_name" : track_name, "track_id" : track_id,"released_date" : released_date,"explict" : explict,"genres" : genres}
                audio_features = sp.audio_features(track_id)[0]
                for feature in columns[7:]:
                    if audio_features is None:
                        print(f"Skipping: {track_name} due to no audio features")
                        break
                    new_row[feature] = audio_features[feature]
                if audio_features is not None:
                    metadata_albums = metadata_albums.append(new_row, ignore_index=True)
            else:
                print(f"Skipping: {track_name} due to a duplicate")

Skipping: Together We Run due to a duplicate
Skipping: After Glow due to a duplicate
Skipping: Let It Rain due to a duplicate
Skipping: Holdin On due to a duplicate
Skipping: Don't Go due to a duplicate
Skipping: United We Stand due to a duplicate
Skipping: Life Rolls On due to a duplicate
Skipping: Together We Run due to a duplicate
Skipping: Don't Give Up On Us due to a duplicate
Skipping: Still Believe In Love due to a duplicate
Skipping: You Got The Best Of Me due to a duplicate
Skipping: Live To Love Again due to a duplicate
Skipping: The Way We Used To Be due to a duplicate
Skipping: Come Away With Me due to a duplicate
Skipping: After Glow due to a duplicate
Skipping: Let It Rain due to a duplicate
Skipping: Holdin On due to a duplicate
Skipping: All Day And All Night due to a duplicate
Skipping: Don't Go due to a duplicate
Skipping: United We Stand due to a duplicate
Skipping: Life Rolls On due to a duplicate
Skipping: Beautiful As You Are due to a duplicate
Skipping: Of a Life

In [15]:
metadata_albums.head()

,artist,album_name,track_name,track_id,released_date,explict,genres,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Journey,Live in Concert at Lollapalooza,Separate Ways - Live,6jNORZ2w4X7SEgs2WBMqxk,2022-12-09,False,"[album rock, classic rock, hard rock, mellow g...",0.336,0.845,3,-7.068,0,0.0541,0.000381,0.000076,0.791,0.2310,133.303,341365,4
1,Journey,Live in Concert at Lollapalooza,Only the Young - Live,1TRHkKFBBayQeFaZpl3Cid,2022-12-09,False,"[album rock, classic rock, hard rock, mellow g...",0.367,0.937,11,-7.174,1,0.0585,0.000020,0.000222,0.706,0.3120,145.956,260419,4
2,Journey,Live in Concert at Lollapalooza,Guitar Interlude - Live,0iUxuyvEWvESgjqhq1tM0P,2022-12-09,False,"[album rock, classic rock, hard rock, mellow g...",0.110,0.834,11,-10.935,1,0.0686,0.200000,0.833000,0.357,0.0387,152.578,70141,4
3,Journey,Live in Concert at Lollapalooza,Stone in Love - Live,1froIKSwt4bA2erWFRV4ve,2022-12-09,False,"[album rock, classic rock, hard rock, mellow g...",0.382,0.941,6,-7.062,1,0.0768,0.000092,0.000295,0.723,0.2640,120.776,318664,4
4,Journey,Live in Concert at Lollapalooza,Be Good to Yourself - Live,6YMJXlWV7upQX4TabuWHJZ,2022-12-09,False,"[album rock, classic rock, hard rock, mellow g...",0.341,0.907,11,-6.648,1,0.0631,0.000206,0.000040,0.919,0.3220,147.978,281984,4


In [16]:
metadata_albums.to_csv(os.path.join(full_path,"metadata_albums_2.csv"))

Get top tracks from artists

In [11]:
import warnings
warnings.filterwarnings('ignore')

columns = ["artist","album_name","track_name","track_id","released_date","explicit","genres","danceability","energy","key","loudness","mode","speechiness","acousticness","instrumentalness","liveness","valence","tempo","duration_ms","time_signature"]

metadata_toptracks = pd.DataFrame(columns=columns)

for artist_uri in artists:
    artist = sp.artist(artist_uri)["name"]
    results = sp.artist_top_tracks(artist_uri,country="NZ") 
    for track in results["tracks"]:
        album_name = track["album"]["name"]
        track_name = track["name"]
        if not (
            metadata_toptracks['artist'].str.contains(artist, regex=False).any() & 
            metadata_toptracks['album_name'].str.contains(album_name, regex=False).any() & 
            metadata_toptracks['track_name'].str.contains(track_name, regex=False).any()):
            
            track_id = track["id"]
            released_date = track["album"]["release_date"]
            explicit = track["explicit"]
            genres = sp.artist(artist_uri)["genres"]
            new_row = {"artist" : artist, "album_name" : album_name, "track_name" : track_name, "track_id" : track_id,"released_date" : released_date, "explicit" : explicit, "genres" : genres}
            audio_features = sp.audio_features(track_id)[0]
            for feature in columns[7:]:
                if audio_features is None:
                    print(f"Skipping: {track_name} due to no audio features")
                    break
                new_row[feature] = audio_features[feature]
            if audio_features is not None:
                metadata_toptracks = metadata_toptracks.append(new_row, ignore_index=True)
        else:
            print(f"Skipping: {track_name} due to a duplicate")

In [12]:
metadata_toptracks.head()

,artist,album_name,track_name,track_id,released_date,explicit,genres,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Journey,Escape (Bonus Track Version),Don't Stop Believin',4bHsxqR3GMrXTxEPLuK5ue,1981,False,"[album rock, classic rock, hard rock, mellow g...",0.500,0.748,4,-9.072,1,0.0363,0.12700,0,0.4470,0.514,118.852,250987,4
1,Journey,Departure,Any Way You Want It,71SvEDmsOwIWw1IozsZoMA,1980,False,"[album rock, classic rock, hard rock, mellow g...",0.529,0.932,7,-7.501,1,0.0488,0.00251,0.00109,0.1360,0.571,138.158,201693,4
2,Journey,Frontiers,Separate Ways (Worlds Apart),1pTw2cNrp9L3esxLAvWnN2,1983,False,"[album rock, classic rock, hard rock, mellow g...",0.467,0.943,0,-5.626,1,0.0576,0.01890,0.000005,0.2160,0.453,131.536,323667,4
3,Journey,Escape (Bonus Track Version),Open Arms,2w4EpqGasrz9qdTwocx54t,1981,False,"[album rock, classic rock, hard rock, mellow g...",0.405,0.210,2,-13.214,1,0.0279,0.58400,0,0.1390,0.174,100.577,202173,3
4,Journey,Frontiers,Faithfully,0l5TpsCL1ObiTEsHeWA0by,1983,False,"[album rock, classic rock, hard rock, mellow g...",0.417,0.573,11,-7.330,1,0.0279,0.08010,0,0.0845,0.290,129.559,266973,4
